# Prediction of sales

### Problem Statement
[The dataset](https://drive.google.com/file/d/1B07fvYosBNdIwlZxSmxDfeAf9KaygX89/view?usp=sharing) represents sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store are available. The aim is to build a predictive model and find out the sales of each product at a particular store.

|Variable|Description|
|: ------------- |:-------------|
|Item_Identifier|Unique product ID|
|Item_Weight|Weight of product|
|Item_Fat_Content|Whether the product is low fat or not|
|Item_Visibility|The % of total display area of all products in a store allocated to the particular product|
|Item_Type|The category to which the product belongs|
|Item_MRP|Maximum Retail Price (list price) of the product|
|Outlet_Identifier|Unique store ID|
|Outlet_Establishment_Year|The year in which store was established|
|Outlet_Size|The size of the store in terms of ground area covered|
|Outlet_Location_Type|The type of city in which the store is located|
|Outlet_Type|Whether the outlet is just a grocery store or some sort of supermarket|
|Item_Outlet_Sales|Sales of the product in the particulat store. This is the outcome variable to be predicted.|

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.



### Explore the problem in following stages:

1. Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome
2. Data Exploration – looking at categorical and continuous feature summaries and making inferences about the data.
3. Data Cleaning – imputing missing values in the data and checking for outliers
4. Feature Engineering – modifying existing variables and creating new ones for analysis
5. Model Building – making predictive models on the data

In [132]:
import pandas as pd
import numpy as np

#Read files:
data = pd.read_csv("C:/Users/Tim/Desktop/lighthouse/w3/d3 - data exploration, cleaning/data_preparation_exercise/regression_exercise.csv", delimiter=',')

In [133]:
n_missing = data.isnull().sum().sort_values(ascending=False)
p_missing = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing = pd.concat([n_missing, p_missing],axis=1, keys = ['number','percent'])
missing

,number,percent
Outlet_Size,2410,0.282764
Item_Weight,1463,0.171653
Item_Outlet_Sales,0,0.000000
Outlet_Type,0,0.000000
Outlet_Location_Type,0,0.000000
Outlet_Establishment_Year,0,0.000000
Outlet_Identifier,0,0.000000
Item_MRP,0,0.000000
Item_Type,0,0.000000
Item_Visibility,0,0.000000


In [134]:
cols_with_missing = missing[missing.percent > 0].index.tolist()
data.dtypes[cols_with_missing]

Outlet_Size     object
Item_Weight    float64
dtype: object

In [135]:
data['Item_Weight'] = data.groupby("Item_Type").transform(lambda x: x.fillna(x.mean))

In [136]:
data['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [137]:
data['Outlet_Size']=data['Outlet_Size'].fillna("Empty")

## 4. Feature Engineering

1. Resolving the issues in the data to make it ready for the analysis.
2. Create some new variables using the existing ones.





### Create a broad category of Type of Item

Item_Type variable has many categories which might prove to be very useful in analysis. Look at the Item_Identifier, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. Use the Item_Identifier variable to create a new column

In [138]:
data['Item_Identifier'].head()

0    FDA15
1    DRC01
2    FDN15
3    FDX07
4    NCD19
Name: Item_Identifier, dtype: object

In [139]:
y = data['Item_Identifier']

In [140]:
# def labelcat(y):
#     if y.str.contains("FD",regex = False):
#         return "FD"
#     if y.str.contains("DR",regex = False):
#         return "DR"
#     return "NC"

def labelcat(y):
    for i in y:
        if "FD" in i:
            return "FD"
        if "DR" in i:
            return "DR"
        return "NC"
    
data.apply(lambda y: labelcat(y), axis=1)

0       FD
1       DR
2       FD
3       FD
4       NC
        ..
8518    FD
8519    FD
8520    NC
8521    FD
8522    DR
Length: 8523, dtype: object

In [141]:
data['Broad_Category'] = data.apply(lambda y: labelcat(y), axis=1)

In [142]:
data['Broad_Category'].head()

0    FD
1    DR
2    FD
3    FD
4    NC
Name: Broad_Category, dtype: object

### Determine the years of operation of a store

Make a new column depicting the years of operation of a store (i.e. how long the store exists). 

In [143]:
data.dtypes

Item_Identifier               object
Item_Weight                   object
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
Broad_Category                object
dtype: object

In [144]:
data['Outlet_Establishment_Year'].head()

0    1999
1    2009
2    1999
3    1998
4    1987
Name: Outlet_Establishment_Year, dtype: int64

In [145]:
from datetime import datetime

In [146]:
currentyear = datetime.now().year
currentyear

2021

In [147]:
data['YoO'] = currentyear - data['Outlet_Establishment_Year']
data['YoO']

0       22
1       12
2       22
3       23
4       34
        ..
8518    34
8519    19
8520    17
8521    12
8522    24
Name: YoO, Length: 8523, dtype: int64

In [148]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Broad_Category,YoO
0,FDA15,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,22
1,DRC01,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,12
2,FDN15,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,22
3,FDX07,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Empty,Tier 3,Grocery Store,732.3800,FD,23
4,NCD19,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,34


### Modify categories of Item_Fat_Content

There are difference in representation in categories of Item_Fat_Content variable. This should be corrected.

In [149]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [150]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({
    'low fat':'Low Fat',
    'LF':'Low Fat',
    'reg':'Regular'
})

In [151]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [152]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Broad_Category,YoO
0,FDA15,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,22
1,DRC01,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,12
2,FDN15,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,22
3,FDX07,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Empty,Tier 3,Grocery Store,732.3800,FD,23
4,NCD19,NCD19,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,34


There are some non-consumables as well and a fat-content should not be specified for them. Create a separate category for such kind of observations.

In [153]:
y = data['Item_Identifier']

In [158]:
# for i in data:
#     if "NC" in data['Item_Identifier']:
#         data['Item_Fat_Content',i] = data['Item_Fat_Content',i].replace(['Low Fat','Regular'],'NC')

data["Item_Fat_Content"].loc[data['Broad_Category']=="NC"]="NC"

C:\Users\Tim\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [159]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'NC'], dtype=object)

In [160]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Broad_Category,YoO
0,FDA15,FDA15,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,FD,22
1,DRC01,DRC01,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,DR,12
2,FDN15,FDN15,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,FD,22
3,FDX07,FDX07,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Empty,Tier 3,Grocery Store,732.3800,FD,23
4,NCD19,NCD19,NC,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,NC,34


### Numerical and One-Hot Encoding of Categorical variables

Since scikit-learn algorithms accept only numerical variables, we need to convert all categorical variables into numeric types. 

- if the variable is Ordinal we can simply map its values into numbers
- if the variable is Nominal (we cannot sort the values) we need to One-Hot Encode them --> create dummy variables

In [189]:
import copy

In [190]:
data2 = copy.deepcopy(data)

In [184]:
data2["Outlet_Size"] = data2["Outlet_Size"].replace({
    "Empty":"0",
    "Small":"1",
    "Medium":"2",
    "High":"3"
})

In [197]:
data2 = data2.join(pd.get_dummies(data2[["Item_Type","Item_Fat_Content","Outlet_Location_Type","Outlet_Type","Broad_Category"]])) #[["Item_Type","Item_Fat_Content","Outlet_Location_Type","Outlet_Type","Broad_Category"]]

In [199]:
data2.dtypes

Item_Identifier                     object
Item_Weight                         object
Item_Fat_Content                    object
Item_Visibility                    float64
Item_Type                           object
Item_MRP                           float64
Outlet_Identifier                   object
Outlet_Establishment_Year            int64
Outlet_Size                         object
Outlet_Location_Type                object
Outlet_Type                         object
Item_Outlet_Sales                  float64
Broad_Category                      object
YoO                                  int64
Item_Type_Baking Goods               uint8
Item_Type_Breads                     uint8
Item_Type_Breakfast                  uint8
Item_Type_Canned                     uint8
Item_Type_Dairy                      uint8
Item_Type_Frozen Foods               uint8
Item_Type_Fruits and Vegetables      uint8
Item_Type_Hard Drinks                uint8
Item_Type_Health and Hygiene         uint8
Item_Type_H

All variables should be by now numeric

### Exporting Data
you can save the processed data to disk

In [205]:
data2.to_csv("C:/Users/Tim/Desktop/lighthouse/w3/d3 - data exploration, cleaning/data_preparation_exercise/datafeng.csv",index=None,header=True)